# 02 - Kubeflow Training with Feast Integration

![Workflow](../docs/02-training-workflow.png)

**Entity DF → Feast get_historical_features → PyTorch DDP → MLflow**

**Prerequisites:** `01-feast-features.ipynb` completed, MLflow running.

In [ ]:
%pip install -q kubeflow-training kubernetes "mlflow>=3.0"
from datetime import datetime
import os

## Configuration

In [ ]:
NAMESPACE = "feast-trainer-demo"
PVC = "feast-pvc"
RUNTIME = "torch-distributed"
MLFLOW_URI = f"http://mlflow.{NAMESPACE}.svc.cluster.local:5000"
EPOCHS = 50
USE_RAY = True
RAY_CLUSTER = "feast-ray"

## Auth

In [ ]:
K8S_TOKEN = os.getenv("K8S_TOKEN", "")
K8S_API = os.getenv("K8S_API_SERVER", "")

In [ ]:
from kubernetes import client as k8s
from kubeflow.training import TrainerClient, CustomTrainer
from kubeflow.training.types import KubernetesBackendConfig, PodTemplateOverrides, PodTemplateOverride, PodSpecOverride, ContainerOverride, Labels

cfg = k8s.Configuration()
if K8S_TOKEN:
    cfg.host = K8S_API
    cfg.verify_ssl = False
    cfg.api_key = {"authorization": f"Bearer {K8S_TOKEN}"}

trainer = TrainerClient(KubernetesBackendConfig(namespace=NAMESPACE, client_configuration=cfg if K8S_TOKEN else None))
runtime = trainer.get_runtime(RUNTIME)
print(f"Runtime: {RUNTIME}")

## Training Function

In [ ]:
def train_fn(params):
    import os, json, numpy as np, pandas as pd, torch, torch.nn as nn, torch.distributed as dist, joblib, mlflow, shutil, time
    from torch.nn.parallel import DistributedDataParallel as DDP
    from torch.utils.data import DataLoader, Dataset, DistributedSampler
    from sklearn.preprocessing import StandardScaler
    from datetime import datetime, timezone, timedelta
    
    dist.init_process_group(backend='nccl' if torch.cuda.is_available() else 'gloo')
    rank, world = dist.get_rank(), dist.get_world_size()
    device = torch.device(f"cuda:{int(os.environ.get('LOCAL_RANK', 0))}" if torch.cuda.is_available() else "cpu")
    print(f"[Rank {rank}] Device: {device}")
    
    OUT = params.get('output_dir', '/shared/models')
    REPO = params.get('feature_repo', '/shared/feature_repo')
    EPOCHS = params.get('epochs', 50)
    
    class MLP(nn.Module):
        def __init__(self, inp, hidden=[512, 256, 128, 64], drop=0.3):
            super().__init__()
            layers = []
            for h in hidden:
                layers.extend([nn.Linear(inp, h), nn.BatchNorm1d(h), nn.ReLU(), nn.Dropout(drop)])
                inp = h
            layers.append(nn.Linear(inp, 1))
            self.net = nn.Sequential(*layers)
        def forward(self, x):
            return self.net(x).squeeze(-1)
    
    class DS(Dataset):
        def __init__(self, X, y):
            self.X = torch.tensor(X, dtype=torch.float32)
            self.y = torch.tensor(y, dtype=torch.float32)
        def __len__(self): return len(self.X)
        def __getitem__(self, i): return self.X[i], self.y[i]
    
    if rank == 0:
        os.makedirs(OUT, exist_ok=True)
        mlflow.set_tracking_uri(os.getenv('MLFLOW_TRACKING_URI', 'http://mlflow:5000'))
        mlflow.set_experiment('sales-forecasting')
        mlflow.start_run(run_name=f"train-{datetime.now().strftime('%Y%m%d-%H%M%S')}")
        
        # Use Ray config for Feast
        if params.get('use_ray'):
            os.environ['FEAST_RAY_USE_KUBERAY'] = 'true'
            os.environ['FEAST_RAY_CLUSTER_NAME'] = params.get('ray_cluster', 'feast-ray')
            os.environ['FEAST_RAY_NAMESPACE'] = params.get('namespace', 'feast-trainer-demo')
            os.environ['FEAST_RAY_SKIP_TLS'] = 'true'
            token_path = '/var/run/secrets/kubernetes.io/serviceaccount/token'
            if os.path.exists(token_path):
                with open(token_path) as f:
                    os.environ['FEAST_RAY_AUTH_TOKEN'] = f.read()
                os.environ['FEAST_RAY_AUTH_SERVER'] = f"https://{os.environ.get('KUBERNETES_SERVICE_HOST')}:{os.environ.get('KUBERNETES_SERVICE_PORT')}"
            ray_cfg = f"{REPO}/feature_store_ray.yaml"
            if os.path.exists(ray_cfg):
                shutil.copy(ray_cfg, f"{REPO}/feature_store.yaml")
        
        from feast import FeatureStore
        store = FeatureStore(repo_path=REPO)
        
        # Entity DF
        entity_rows = [{'store_id': s, 'dept_id': d, 'event_timestamp': datetime(2022,1,1,tzinfo=timezone.utc) + timedelta(weeks=w)}
            for w in range(104) for s in range(1, 46) for d in range(1, 15)]
        entity_df = pd.DataFrame(entity_rows)
        print(f"Entity DF: {len(entity_df):,} rows")
        
        t0 = time.time()
        df = store.get_historical_features(entity_df=entity_df, features=store.get_feature_service('training_features')).to_df()
        print(f"✅ Feast: {len(df):,} rows in {time.time()-t0:.1f}s")
        
        df = df.dropna(subset=['weekly_sales']).sort_values('event_timestamp')
        split = int(len(df) * 0.8)
        train_df, val_df = df.iloc[:split], df.iloc[split:]
        
        exclude = ['store_id', 'dept_id', 'event_timestamp', 'date', 'weekly_sales']
        feat_cols = [c for c in df.columns if c not in exclude and df[c].dtype in [np.float64, np.int64, np.float32, np.int32]]
        print(f"Features ({len(feat_cols)}): {feat_cols[:5]}...")
        
        X_train, y_train = train_df[feat_cols].fillna(0).values, train_df['weekly_sales'].values
        X_val, y_val = val_df[feat_cols].fillna(0).values, val_df['weekly_sales'].values
        
        scaler, y_scaler = StandardScaler(), StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_val = scaler.transform(X_val)
        y_train_s = y_scaler.fit_transform(np.log1p(y_train).reshape(-1,1)).flatten()
        y_val_s = y_scaler.transform(np.log1p(y_val).reshape(-1,1)).flatten()
        
        joblib.dump({'scaler_X': scaler, 'scaler_y': y_scaler, 'use_log_transform': True}, f'{OUT}/scalers.joblib')
        joblib.dump(feat_cols, f'{OUT}/feature_cols.pkl')
        np.savez(f'{OUT}/.data.npz', X_train=X_train, y_train=y_train_s, X_val=X_val, y_val=y_val_s, y_val_orig=y_val)
        np.save(f'{OUT}/.dim.npy', [X_train.shape[1]])
        mlflow.log_params({'epochs': EPOCHS, 'train_rows': len(train_df), 'val_rows': len(val_df), 'features': len(feat_cols)})
    
    dist.barrier()
    data = np.load(f'{OUT}/.data.npz')
    X_train, y_train, X_val, y_val, y_val_orig = data['X_train'], data['y_train'], data['X_val'], data['y_val'], data['y_val_orig']
    inp_dim = int(np.load(f'{OUT}/.dim.npy')[0])
    dist.barrier()
    
    train_ds = DS(X_train, y_train)
    val_ds = DS(X_val, y_val)
    sampler = DistributedSampler(train_ds, num_replicas=world, rank=rank)
    train_loader = DataLoader(train_ds, batch_size=64, sampler=sampler)
    val_loader = DataLoader(val_ds, batch_size=64)
    
    model = DDP(MLP(inp_dim).to(device), device_ids=[int(os.environ.get('LOCAL_RANK',0))] if torch.cuda.is_available() else None)
    opt = torch.optim.AdamW(model.parameters(), lr=5e-4, weight_decay=1e-3)
    sched = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, factor=0.5, patience=3)
    crit = nn.MSELoss()
    
    best_loss, best_mape = float('inf'), float('inf')
    for ep in range(EPOCHS):
        sampler.set_epoch(ep)
        model.train()
        train_loss = 0.0
        for X_b, y_b in train_loader:
            X_b, y_b = X_b.to(device), y_b.to(device)
            opt.zero_grad()
            loss = crit(model(X_b), y_b)
            loss.backward()
            opt.step()
            train_loss += loss.item()
        train_loss /= len(train_loader)
        
        model.eval()
        with torch.no_grad():
            preds = np.concatenate([model(X.to(device)).cpu().numpy() for X, _ in val_loader])
        val_loss = np.mean((preds - y_val)**2)
        sched.step(val_loss)
        
        if rank == 0:
            y_sc = joblib.load(f'{OUT}/scalers.joblib')['scaler_y']
            pred_orig = np.expm1(y_sc.inverse_transform(preds.reshape(-1,1)).flatten())
            mask = y_val_orig > 1000
            mape = np.mean(np.abs((y_val_orig[mask] - pred_orig[mask]) / y_val_orig[mask])) * 100
            mlflow.log_metrics({'train_loss': train_loss, 'val_loss': val_loss, 'mape': mape, 'lr': opt.param_groups[0]['lr']}, step=ep)
            if (ep + 1) % 10 == 0:
                print(f"Ep {ep+1}/{EPOCHS} | Train: {train_loss:.4f} | Val: {val_loss:.4f} | MAPE: {mape:.1f}%")
            if val_loss < best_loss:
                best_loss, best_mape = val_loss, mape
                torch.save(model.module.state_dict(), f'{OUT}/best_model.pt')
        dist.barrier()
    
    if rank == 0:
        print(f"\n✅ DONE: MAPE {best_mape:.1f}%")
        mlflow.log_metrics({'best_mape': best_mape, 'best_val_loss': best_loss})
        m = MLP(inp_dim)
        m.load_state_dict(torch.load(f'{OUT}/best_model.pt'))
        m.eval()
        mlflow.pytorch.log_model(m, 'model')
        feat_cols = joblib.load(f'{OUT}/feature_cols.pkl')
        json.dump({'model_type': 'SalesMLP', 'input_dim': inp_dim, 'hidden_dims': [512,256,128,64], 'dropout': 0.3, 'best_mape': float(best_mape), 'feature_columns': feat_cols}, open(f'{OUT}/model_metadata.json', 'w'))
        mlflow.end_run()
        for f in ['.data.npz', '.dim.npy']:
            try: os.remove(f'{OUT}/{f}')
            except: pass
    dist.destroy_process_group()

## Submit Job

In [ ]:
job_id = datetime.now().strftime('%m%d-%H%M')
params = {'epochs': EPOCHS, 'use_ray': USE_RAY, 'ray_cluster': RAY_CLUSTER, 'namespace': NAMESPACE, 'output_dir': '/shared/models', 'feature_repo': '/shared/feature_repo'}

job = trainer.train(
    trainer=CustomTrainer(
        func=train_fn,
        num_nodes=1,
        resources_per_node={'cpu': 4, 'memory': '8Gi', 'nvidia.com/gpu': 1},
        packages_to_install=['feast[ray,postgres]==0.59.0', 'codeflare-sdk', 'psycopg2-binary', 'scikit-learn', 'pandas', 'pyarrow', 'joblib', 'mlflow>=3.0'],
        env={'MLFLOW_TRACKING_URI': MLFLOW_URI}
    ),
    runtime=runtime,
    parameters=params,
    options=[
        Labels({'app': 'sales-forecasting'}),
        PodTemplateOverrides(PodTemplateOverride(
            target_jobs=['node'],
            spec=PodSpecOverride(
                volumes=[{'name': 'shared', 'persistentVolumeClaim': {'claimName': PVC}}],
                containers=[ContainerOverride(name='node', volume_mounts=[{'name': 'shared', 'mountPath': '/shared'}])]
            )
        ))
    ]
)
print(f"✅ Submitted: {job}")

## Monitor

In [ ]:
trainer.wait_for_job_status(name=job, status={'Running'}, timeout=300)
_ = trainer.get_job_logs(name=job, follow=True)

In [ ]:
trainer.wait_for_job_status(name=job, status={'Complete', 'Failed'}, timeout=3600)

## MLflow Results

In [ ]:
import mlflow
mlflow.set_tracking_uri(MLFLOW_URI)
exp = mlflow.get_experiment_by_name('sales-forecasting')
if exp:
    runs = mlflow.search_runs([exp.experiment_id], max_results=5, order_by=['start_time DESC'])
    display(runs[['tags.mlflow.runName', 'metrics.best_mape']])
else:
    print("Experiment not found yet")

**Next:** `03-inference.ipynb` (deploy KServe first: `kubectl apply -f ../manifests/08-kserve-inference.yaml`)